In [1]:
import csv
import re
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import nltk
nltk.download('punkt')
import time

import pandas as pd
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KIMWONGYU\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def load_and_preprocess(filename):
    #./data_toeic/5000+2000.csv
    f = open(filename, 'r',encoding='utf-8')
    csvReader = csv.reader(f)

    raw_data = []
    sentences = []
    label = []
    for row in csvReader:
        raw_data.append(row)
    f.close()

    del raw_data[0]
    
    print("총 문제 개수: ", len(raw_data))
    print("총 문장 개수: ", len(raw_data)*4)
    for row_index, row in enumerate(raw_data):
        row_sentence = []
        row_label = []
        hype1 = 0
        hype2 = 0
        for item_index, item in enumerate(row):
            # .뒤에 나오는 것 다 없애기 .이 여러 개 있을지 모르니 마지막 .을 이용하기. 두 문장인 경우 .과 ?과 !이 존재
            # 특수문자 앞에 공백으로 하기
            # 공백 없애기
            # 공백 두개, 세개 -> 한개로 바꾸기
            # _____,----- 연달아 있을시 index 찾아 양쪽 공백 만들기
            # 소문자로 바꾸기

            if item_index == 0:
                #
                index_list = []
                index_of_dot = item.rfind('.')
                index_of_question = item.rfind('?')
                index_of_surprise = item.rfind('!')
                index_list.append(index_of_dot)
                index_list.append(index_of_question)
                index_list.append(index_of_surprise)
                standard = max(index_list)+1
                if standard >0:
                    item = item[:standard]   

                #
                item = item.strip()
                #
                item = item.replace("  ", " ")
                item = item.replace("  ", " ")

                #
                index_of_hype1 = item.find('__')
                index_of_hype2 = item.find('--')
                index_of_hype3 = item.rfind('__')
                index_of_hype4 = item.rfind('--')
                if index_of_hype1 > index_of_hype2:
                    if index_of_hype3 > index_of_hype4:
                        hype1 = index_of_hype1
                        hype2 = index_of_hype3+1
                elif index_of_hype2 > index_of_hype1:
                    if index_of_hype4 > index_of_hype3:
                        hype1 = index_of_hype2
                        hype2 = index_of_hype4+1

                #----- => hype1 = 0 , hype2 = 4
                if hype1 == 0:
                    if (hype2+1) < len(item):
                        if item[hype2+1] != ' ':
                            item = item[:hype2+1] + ' ' + item[hype2+1:]
                else:
                    if item[hype1-1] != ' ':
                        item = item[:hype1] + ' ' + item[hype1:]
                        hype1 = hype1+1
                        hype2 = hype2+1
                    if (hype2+1) < len(item):
                        if item[hype2+1] != ' ':
                            item = item[:hype2+1] + ' ' + item[hype2+1:]
                #
                item = item.lower()
                # 문장이 아닐 때
            else:
                 item = item.strip()
            raw_data[row_index][item_index] = item
       
        #sentence 4개 만들기
        row[0] = row[0].replace(row[0][hype1:hype2+1],"")
        for i in range(1,5):
            sentence = row[0][:hype1] + row[i] + row[0][hype1:]
            row_sentence.append(sentence)

        sentences.append(row_sentence)

        #label 만들기
        count = 0
        for i in range(1,5):
            if row[5] == row[i]:
                row_label.append(1)
                count+=1
            else:
                row_label.append(0)

        #모두 0일 경우 찾아서 1 넣어 주기
        if count!=1:
            if 'A' in row[5]:
                row_label[0]=1
            elif 'B' in row[5]:
                row_label[1]=1
            elif 'C' in row[5]:
                row_label[2]=1
            elif 'D' in row[5]:
                row_label[3]=1

        #레이블에 추가해주기
        label.append(row_label)

    sentences= np.array(sentences)
    sentences= sentences.flatten()
    label = np.array(label)
    label = label.flatten()
    
    return sentences, label

In [3]:
def pre_process_test(sentence,a1,a2,a3,a4):

    raw_data = []
    sentences = []
    
    temp =[]
    temp.append(sentence)
    temp.append(a1)
    temp.append(a2)
    temp.append(a3)
    temp.append(a4)
    raw_data.append(temp)
    
    print("총 문제 개수: ", len(raw_data))
    print("총 문장 개수: ", len(raw_data)*4)
    for row_index, row in enumerate(raw_data):
        row_sentence = []
        row_label = []
        hype1 = 0
        hype2 = 0
        for item_index, item in enumerate(row):
            # .뒤에 나오는 것 다 없애기 .이 여러 개 있을지 모르니 마지막 .을 이용하기. 두 문장인 경우 .과 ?과 !이 존재
            # 특수문자 앞에 공백으로 하기
            # 공백 없애기
            # 공백 두개, 세개 -> 한개로 바꾸기
            # _____,----- 연달아 있을시 index 찾아 양쪽 공백 만들기
            # 소문자로 바꾸기

            if item_index == 0:
                #
                index_list = []
                index_of_dot = item.rfind('.')
                index_of_question = item.rfind('?')
                index_of_surprise = item.rfind('!')
                index_list.append(index_of_dot)
                index_list.append(index_of_question)
                index_list.append(index_of_surprise)
                standard = max(index_list)+1
                if standard >0:
                    item = item[:standard]   

                #
                item = item.strip()
                #
                item = item.replace("  ", " ")
                item = item.replace("  ", " ")

                #
                index_of_hype1 = item.find('__')
                index_of_hype2 = item.find('--')
                index_of_hype3 = item.rfind('__')
                index_of_hype4 = item.rfind('--')
                if index_of_hype1 > index_of_hype2:
                    if index_of_hype3 > index_of_hype4:
                        hype1 = index_of_hype1
                        hype2 = index_of_hype3+1
                elif index_of_hype2 > index_of_hype1:
                    if index_of_hype4 > index_of_hype3:
                        hype1 = index_of_hype2
                        hype2 = index_of_hype4+1

                #----- => hype1 = 0 , hype2 = 4
                if hype1 == 0:
                    if (hype2+1) < len(item):
                        if item[hype2+1] != ' ':
                            item = item[:hype2+1] + ' ' + item[hype2+1:]
                else:
                    if item[hype1-1] != ' ':
                        item = item[:hype1] + ' ' + item[hype1:]
                        hype1 = hype1+1
                        hype2 = hype2+1
                    if (hype2+1) < len(item):
                        if item[hype2+1] != ' ':
                            item = item[:hype2+1] + ' ' + item[hype2+1:]
                #
                item = item.lower()
                # 문장이 아닐 때
            else:
                 item = item.strip()
            raw_data[row_index][item_index] = item
        #sentence 4개 만들기
        row[0] = row[0].replace(row[0][hype1:hype2+1],"")
        for i in range(1,5):
            sentence = row[0][:hype1] + row[i] + row[0][hype1:]
            row_sentence.append(sentence)

        sentences.append(row_sentence)

    sentences= np.array(sentences)
    sentences= sentences.flatten()
    
    return sentences

In [20]:
def load_example(filename):
    df = pd.read_csv(filename, error_bad_lines=False)
    sentences = df['SentimentText'][:32000]
    label = df['Sentiment'][:32000]
    R_S = []
    R_L = []
    
    for s in sentences:
        R_S.append(s)
    for l in label:
        R_L.append(l)
    R_S_train = R_S[:30000]
    R_L_train = R_L[:30000]
    R_S_test = R_S[30000:32000]
    R_L_test = R_L[30000:32000]
    print(R_S[:5])
    print(R_L[:5])
    return R_S_train,R_L_train, R_S_test, R_L_test

In [5]:
#가공만 된 문장 indexing, embedding 전 입니다.
class ToeicDataset(Dataset):
    def __init__(self, sentences, label):
        self.sentences = sentences
        self.label = label
        self.len = len(self.label)
        
        self.label_list = list(sorted(set(self.label)))
        
    def __getitem__(self, index):
        return self.sentences[index], self.label[index]
    def __len__(self):
        return self.len
    
    def get_labels(self):
        return self.label_list
    
    def get_label(self,id):
        return self.label_list[id]
    def get_label_id(self,label):
        return self.label_list.index(label)

In [6]:
def make_vocab(sentences):
    vocab = set()
    for index, sentence in enumerate(sentences):
        temp = nltk.word_tokenize(sentence)
        vocab.update(temp)
    word_to_ix = {word:i+1 for i, word in enumerate(vocab)}
    word_to_ix['_PAD'] = 0
    word_to_ix['_UNK'] = 1
    vocabsize = len(word_to_ix)
    return vocab, vocabsize, word_to_ix

In [7]:
#make torch
def make_variables(sentences, label,vocabulary):
    
    final_sentences = []
    
    #tokenizing
    for index, sentence in enumerate(sentences):
        final_sentences.append(nltk.word_tokenize(sentence))
    
    #indexing
    for index, sentence in enumerate(final_sentences):
        final_sentences[index]=[vocabulary[word] for word in sentence]

    #각자의 seq_length 구하기 (미니배치별로 진행)
    seq_lengths = []
    for sentence in final_sentences:
        seq_lengths.append(len(sentence))
    seq_lengths = torch.LongTensor(seq_lengths)

#     print("패딩전")
#     print(final_sentences[:5])
#     print(seq_lengths[:5])
#     print(label[:5])
    return padding_tensor_sorting(final_sentences,seq_lengths,label)

In [33]:
def padding_tensor_sorting(sentences, seq_lengths, label):
    seq_tensor = torch.zeros((len(sentences), seq_lengths.max())).long()
    print('seq_tensor(max):' , seq_lengths.max())
    for idx, (seq, seq_len) in enumerate(zip(sentences, seq_lengths)):
        seq_tensor[idx, :seq_len] = torch.LongTensor(seq)
    
    if len(seq_tensor) !=1:
        seq_lengths, perm_idx = seq_lengths.sort(0,descending=True)
        seq_tensor = seq_tensor[perm_idx]
    
    
    target = torch.tensor(label, dtype=torch.long)
    
    if len(seq_tensor) !=1 and len(target)!=0:
        target = target[perm_idx]
        
#     print("패딩후")
#     print(seq_tensor[:5])
#     print(seq_lengths[:5])
#     print(target[:5])
    return create_variable(seq_tensor), \
        create_variable(seq_lengths), \
        create_variable(target)

In [9]:
def create_variable(tensor):
    #tensor를 gpu 이용 가능한지
    if torch.cuda.is_available():
        return Variable(tensor.cuda())
    else:
        return Variable(tensor)

In [10]:
def train():
    total_loss = 0
    for i, (sentences, label) in enumerate(train_loader,1):
        input, seq_lengths, target = make_variables(sentences, label,word_to_ix)
        
        output = classifier(input, seq_lengths)
        
        loss = criterion(output, target)
        
        total_loss +=loss.data[0]
        
        classifier.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i%10 == 0:
            print("output, target", output, target)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.2f}'.format(
                epoch,  i *
                len(sentences), len(train_loader.dataset),
                100. * i * len(sentences) / len(train_loader.dataset),
                total_loss / i * len(sentences)))
        
    return total_loss

In [11]:
# def validation_make_vocab(sentences_test):
#     vocab = set()
#     for index, sentence in enumerate(sentences_test):
#         temp = nltk.word_tokenize(sentence)
#         vocab.update(temp)
#     word_to_ix2 = word_to_ix
    
#     length = len(word_to_ix2)
    
#     for i,word in enumerate(vocab,length):
#         if word not in word_to_ix2:
#             word_to_ix2[i]=word
    
#     vocabsize = len(word_to_ix2)
#     return vocab,vocabsize,word_to_ix2

In [12]:
def validation():
    correct =0
    test_data_size = len(test_loader.dataset)
    
    for i,(sentences, label) in enumerate(test_loader,1):
        
        input, seq_lengths, target = make_variables(sentences, label,word_to_ix_v)
        
        output = valid_model(input, seq_lengths)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        print('prediction, ', pred)
        print('target, ',target)
    print('correct: ', correct)
    print('전체 싸이즈: ', test_data_size)
    
#     print('evaluating trained model...')
    
#     input,seq_lengths, target = make_variables([sentences],[])
#     output = classifier(input, seq_lengths)
#     print(output)
#     pred = output.data.max(1, keepdim = True)[1]
#     print(pred)
#     label_id = pred.cpu().numpy()[0][0]
#     print(label_id)
#     print(sentences, " is ", train_dataset.get_label(label_id))

In [24]:
def one_problem_test(sentence, a1,a2,a3,a4):
    
    sentences = pre_process_test(sentence,a1,a2,a3,a4)
    print(sentences)
    #단어집 불러오기
    file = open("vocabulary", "rb")
    vocabulary = pickle.load(file)
    file.close()
    
    vocab = set()
    temp = nltk.word_tokenize(sentence)
    vocab.update(temp)
    
    length = len(vocabulary)
    
    for i, word in enumerate(vocab):
        vocabulary[word]= length+i
    vocabsize_t = len(vocabulary)
    print(vocabsize_t)
    test_model = torch.load('saved_gru')
    test_model.n_vocab = vocabsize_t
    test_model.embed = nn.Embedding(valid_model.n_vocab, valid_model.embed_dim)
    
    input, seq_lengths, target = make_variables(sentences,[],vocabulary)
    
    output = test_model(input,seq_lengths)
    pred = output.data.max(1,keepdim=True)[1]
    print("softmax 직후...", output)
    print("1개만 고른후...", pred)

In [14]:
class myModel(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, bidirectional=True ,dropout_p=0.2):
        super(myModel, self).__init__()
        
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.n_vocab = n_vocab
        self.embed_dim = embed_dim
        self.n_classes = n_classes
        self.n_directions = int(bidirectional) + 1
        
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
#         self.lstm = nn.LSTM(self.embed_dim, self.hidden_dim,
#                             num_layers=self.n_layers,
#                             dropout=dropout_p,
#                             batch_first=True)
        self.lstm = nn.GRU(self.embed_dim, self.hidden_dim,
                            self.n_layers,
                            #dropout=dropout_p,
                            batch_first=True)
#         self.relu = nn.ReLU()
        self.out = nn.Linear(self.hidden_dim, self.n_classes)
#         self.fc1 = nn.Linear(self.hidden_dim, 50)
#         self.fc2 = nn.Linear(50, self.n_classes)
    def forward(self, x, seq_lengths):
        
        x = x.t()
        sen_len = x.size(0)
        batch_size = x.size(1)
        print(self.n_layers, self.hidden_dim, self.n_vocab, self.embed_dim, self.n_classes, self.n_directions)
        
        embedded = self.embed(x)
        
#         print(embedded)
        lstm_input = pack_padded_sequence(embedded, seq_lengths.data.cpu().numpy())
        self.hidden = self._init_hidden(batch_size)
        self.lstm.flatten_parameters()
        
        lstm_out, self.hidden = self.lstm(lstm_input)
        lstm_out, lengths = pad_packed_sequence(lstm_out)
        
#         h_t = self.dropout(self.hidden[-1])
#         logit = self.out(h_t[-1])
        logit = self.out(self.hidden[-1])
#         print(logit)
        return logit
    
    def _init_hidden(self, batch_size):
        hidden = torch.zeros((self.n_layers, self.n_directions,
                             batch_size, self.hidden_dim))
        return create_variable(hidden)

In [21]:
if __name__ =="__main__":
    #
    BATCH_SIZE = 64
    EPOCHS = 20
    learning_rate =0.01
    torch.manual_seed(42)
    
    #load data
#     sentences, label = load_and_preprocess('./data_toeic/5000+2000.csv')
    sentences, label, sentences_test, label_test = load_example('./Sentiment Analysis Dataset.csv')
    train_dataset = ToeicDataset(sentences, label)
    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True)
    test_dataset = ToeicDataset(sentences_test,label_test)
    test_loader = DataLoader(dataset=test_dataset,
                            batch_size=BATCH_SIZE,
                            shuffle=True)
    
    # hyper parameter, user data
    n_layer = 2
    n_classes = 2
    hidden_dim = 100
    embed_dim = 64
    bidirectional = True
    dropout_p = 0.4
    vocab,vocabsize,word_to_ix = make_vocab(sentences)
    sentence_temp = sentences
    sentence_temp +=sentences_test
    vocab_v,vocabsize_v,word_to_ix_v = make_vocab(sentence_temp)
    file=open("vocabulary","wb")
    pickle.dump(word_to_ix, file)
    file.close()
    #make model
    classifier = myModel(n_layer, hidden_dim, vocabsize, embed_dim, n_classes, bidirectional, dropout_p)
    
    #classifier and cuda
    #병렬 처리
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        # dim = 0 [33, xxx] -> [11, ...], [11, ...], [11, ...] on 3 GPUs
        classifier = nn.DataParallel(classifier)
    if torch.cuda.is_available():
        classifier.cuda()
    
    optimizer = torch.optim.Adam(classifier.parameters(),lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(1, 2):
        train()

    


b'Skipping line 8836: expected 4 fields, saw 5\n'
b'Skipping line 535882: expected 4 fields, saw 7\n'


['                     is so sad for my APL friend.............', '                   I missed the New Moon trailer...', '              omg its already 7:30 :O', "          .. Omgaga. Im sooo  im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)...", '         i think mi bf is cheating on me!!!       T_T']
[0, 0, 1, 0, 0]
seq_tensor(max): tensor(47)
2 100 53011 64 2 2
seq_tensor(max): tensor(36)
2 100 53011 64 2 2
seq_tensor(max): 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  # Remove the CWD from sys.path while we load stuff.


tensor(38)
2 100 53011 64 2 2
seq_tensor(max): tensor(36)
2 100 53011 64 2 2
seq_tensor(max): tensor(44)
2 100 53011 64 2 2
seq_tensor(max): tensor(38)
2 100 53011 64 2 2
seq_tensor(max): tensor(34)
2 100 53011 64 2 2
seq_tensor(max): tensor(33)
2 100 53011 64 2 2
seq_tensor(max): tensor(39)
2 100 53011 64 2 2
seq_tensor(max): tensor(43)
2 100 53011 64 2 2
output, target tensor([[ 0.1916,  0.2408],
        [ 0.0360, -0.2476],
        [-0.2439, -0.0805],
        [ 0.2264, -0.2374],
        [ 0.0511,  0.3604],
        [ 0.1443, -0.0599],
        [ 0.0073, -0.2121],
        [-0.0292,  0.1174],
        [-0.0115,  0.1056],
        [-0.3115,  0.2227],
        [-0.3189, -0.0995],
        [ 0.6150, -0.1159],
        [ 0.0584,  0.0905],
        [ 0.2372,  0.1144],
        [-0.3779,  0.3219],
        [-0.1157, -0.2449],
        [-0.1267,  0.0181],
        [-0.3161,  0.5188],
        [-0.0812, -0.3110],
        [ 0.1881,  0.3208],
        [-0.0093,  0.2397],
        [-0.9212,  0.9833],
        [-

seq_tensor(max): tensor(35)
2 100 53011 64 2 2
seq_tensor(max): tensor(43)
2 100 53011 64 2 2
seq_tensor(max): tensor(114)
2 100 53011 64 2 2
seq_tensor(max): tensor(40)
2 100 53011 64 2 2
seq_tensor(max): tensor(44)
2 100 53011 64 2 2
seq_tensor(max): tensor(31)
2 100 53011 64 2 2
seq_tensor(max): tensor(35)
2 100 53011 64 2 2
seq_tensor(max): tensor(36)
2 100 53011 64 2 2
seq_tensor(max): tensor(35)
2 100 53011 64 2 2
output, target tensor([[ 0.0146,  0.0145],
        [-0.8834,  0.5986],
        [ 0.1675,  0.1011],
        [ 0.9621, -0.2095],
        [-0.6687,  0.9001],
        [ 0.8383, -0.3938],
        [-0.3150,  0.1609],
        [-0.4063,  0.1931],
        [ 0.3970, -0.5066],
        [-0.3863,  0.1422],
        [ 0.8264,  0.0698],
        [ 0.0724,  0.3343],
        [-0.0733,  0.0603],
        [ 0.3444, -0.2967],
        [ 0.5517, -0.0845],
        [-0.2821,  0.6814],
        [ 0.2261,  0.2080],
        [ 0.4016,  0.0684],
        [ 0.6643, -0.3061],
        [ 0.0495, -0.0983],
 

...

seq_tensor(max): tensor(40)
2 100 53011 64 2 2
seq_tensor(max): tensor(50)
2 100 53011 64 2 2
seq_tensor(max): tensor(36)
2 100 53011 64 2 2
seq_tensor(max): tensor(38)
2 100 53011 64 2 2
seq_tensor(max): tensor(37)
2 100 53011 64 2 2
seq_tensor(max): tensor(33)
2 100 53011 64 2 2
seq_tensor(max): tensor(36)
2 100 53011 64 2 2
seq_tensor(max): tensor(36)
2 100 53011 64 2 2
seq_tensor(max): tensor(73)
2 100 53011 64 2 2
seq_tensor(max): tensor(36)
2 100 53011 64 2 2
output, target tensor([[ 0.4762, -0.5675],
        [-0.8343,  0.8516],
        [ 0.2324, -0.3033],
        [ 0.3683, -0.2346],
        [ 0.5403, -0.3969],
        [ 0.4412, -0.1431],
        [ 0.6628, -0.5520],
        [ 1.2619, -1.0830],
        [-0.5797,  0.7025],
        [-1.2075,  1.2331],
        [ 0.3116,  0.1362],
        [ 0.8142, -0.7182],
        [ 0.7966, -0.6898],
        [-0.0898,  0.3508],
        [-1.1787,  1.3922],
        [-0.2991,  0.1291],
        [-0.3886,  0.3196],
        [ 0.6761, -0.8863],
        [ 0

seq_tensor(max): tensor(37)
2 100 53011 64 2 2
seq_tensor(max): tensor(32)
2 100 53011 64 2 2
seq_tensor(max): tensor(70)
2 100 53011 64 2 2
output, target tensor([[-0.4967,  0.7923],
        [ 1.9556, -1.3921],
        [ 1.7580, -1.5780],
        [-0.2247,  0.6062],
        [ 1.1795, -0.4905],
        [-0.4051,  0.4553],
        [-0.9253,  1.2011],
        [ 0.3907,  0.0868],
        [-0.9397,  1.2507],
        [-0.6212,  0.4953],
        [ 0.3842, -0.0348],
        [-0.1477,  0.5171],
        [-0.6318,  0.7428],
        [-0.9543,  1.0393],
        [-0.7293,  0.5541],
        [-1.7433,  1.7246],
        [-0.2862,  0.3447],
        [-0.0966, -0.1143],
        [-0.9001,  1.2720],
        [-0.8168,  1.1315],
        [ 0.2308,  0.1249],
        [-0.7500,  0.6964],
        [ 1.1858, -1.3416],
        [ 0.8117, -0.6330],
        [ 0.3823, -0.9231],
        [ 0.5080, -0.1706],
        [-0.0302, -0.1392],
        [ 0.8230, -0.8080],
        [ 1.1024, -0.8106],
        [ 1.2041, -0.7794],
    

seq_tensor(max): tensor(70)
2 100 53011 64 2 2
seq_tensor(max): tensor(42)
2 100 53011 64 2 2
seq_tensor(max): tensor(42)
2 100 53011 64 2 2
seq_tensor(max): tensor(38)
2 100 53011 64 2 2
seq_tensor(max): tensor(35)
2 100 53011 64 2 2
seq_tensor(max): tensor(38)
2 100 53011 64 2 2
seq_tensor(max): tensor(34)
2 100 53011 64 2 2
seq_tensor(max): tensor(35)
2 100 53011 64 2 2
seq_tensor(max): tensor(41)
2 100 53011 64 2 2
output, target tensor([[ 0.1843, -0.4963],
        [ 0.2874, -0.7842],
        [ 0.3631, -0.4533],
        [ 1.0475, -0.8455],
        [-0.7947,  0.3961],
        [-1.2787,  1.4635],
        [ 0.5745, -0.4733],
        [ 0.7319, -0.4303],
        [ 1.0784, -1.1526],
        [ 1.0130, -1.0225],
        [-1.2381,  1.1104],
        [ 0.8341, -0.8150],
        [ 0.0712, -0.0922],
        [ 0.7346, -0.7565],
        [-0.3925,  0.1342],
        [-0.9804,  0.5795],
        [ 0.1000, -0.3035],
        [-0.4893,  0.2906],
        [-1.4162,  1.5466],
        [-1.6483,  1.4608],
  

seq_tensor(max): tensor(39)
2 100 53011 64 2 2
output, target tensor([[-0.6987,  0.5440],
        [ 0.0132, -0.3039],
        [-0.3494,  0.2087],
        [ 0.7419, -0.5365],
        [ 1.1901, -0.8578],
        [ 0.4375, -0.4354],
        [ 0.2596, -0.3420],
        [ 0.0813,  0.0244],
        [-0.5993,  0.2766],
        [ 0.0811, -0.2480],
        [-0.8690,  0.5773],
        [ 0.5680, -0.3779],
        [ 0.6425, -0.6973],
        [ 1.9502, -1.4074],
        [ 0.0698, -0.2103],
        [ 0.2357, -0.0241],
        [-0.7202,  0.6928],
        [ 0.4707, -0.1226],
        [ 0.0192, -0.0572],
        [-0.7044,  0.3637],
        [-1.2363,  1.1249],
        [ 0.3121, -0.5679],
        [-0.2454,  0.0032],
        [-0.8115,  0.6720],
        [-1.5868,  1.3112],
        [-1.1884,  1.2032],
        [-0.0487,  0.1010],
        [-1.3628,  0.9908],
        [-1.5552,  1.1418],
        [-1.9646,  1.3313],
        [ 0.9127, -0.9404],
        [ 0.1164, -0.0969],
        [ 1.2530, -1.0426],
        [-0.36

seq_tensor(max): tensor(63)
2 100 53011 64 2 2
seq_tensor(max): tensor(36)
2 100 53011 64 2 2
seq_tensor(max): tensor(40)
2 100 53011 64 2 2
seq_tensor(max): tensor(39)
2 100 53011 64 2 2
seq_tensor(max): tensor(80)
2 100 53011 64 2 2
seq_tensor(max): tensor(38)
2 100 53011 64 2 2
seq_tensor(max): tensor(42)
2 100 53011 64 2 2


In [22]:
    torch.save(classifier,'saved_gru')

    valid_model = torch.load('saved_gru')
    valid_model.n_vocab = vocabsize_v
    valid_model.embed = nn.Embedding(valid_model.n_vocab, valid_model.embed_dim)
    print('validation!!!!')
    print(len(vocab))
    print(vocabsize)
    print(len(word_to_ix))
    print(len(vocab_v))
    print(vocabsize_v)
    print(len(word_to_ix_v))
    print(valid_model)
    print(valid_model.n_vocab)
    validation()


C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:193: UserWarning: Couldn't retrieve source code for container of type myModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


validation!!!!
53009
53011
53011
55633
55635
55635
myModel(
  (embed): Embedding(55635, 64)
  (dropout): Dropout(p=0.4)
  (lstm): GRU(64, 100, num_layers=2, batch_first=True)
  (out): Linear(in_features=100, out_features=2, bias=True)
)
55635
seq_tensor(max): tensor(36)
2 100 55635 64 2 2
prediction,  tensor([[ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 1],
        [ 0],
        [ 0],
        [ 1],
        [ 1],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 1],
        [ 0],
        [ 1

prediction,  tensor([[ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 1],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0]])
target,  tensor([ 1,  0,  1,  1,  1,  0,  0,  0,  1,  1,  1,  1,  1,  1,
         0,  0,  

prediction,  tensor([[ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 1],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 1],
        [ 0],
        [ 1],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0]])
target,  tensor([ 1,  1,  0,  0,  0,  0,  1,  0,  1,  1,  1,  0,  1,  1,
         0,  1,  

In [41]:
    one_problem_test('____________ harvesting techniques have been instrumental in helping many of the local small farmers to become more self-sufficient. ',
                    ' improve', 'improved', 'improvement', 'improves')

총 문제 개수:  1
총 문장 개수:  4
['improve harvesting techniques have been instrumental in helping many of the local small farmers to become more self-sufficient.'
 'improved harvesting techniques have been instrumental in helping many of the local small farmers to become more self-sufficient.'
 'improvement harvesting techniques have been instrumental in helping many of the local small farmers to become more self-sufficient.'
 'improves harvesting techniques have been instrumental in helping many of the local small farmers to become more self-sufficient.']
53015
seq_tensor(max): tensor(19)
tensor([ 31745,  53020,  53018,  53016,  53019,  53029,  53021,  53015,
         53022,  53017,  53014,  53023,  53028,  53012,  53024,  53025,
         53027,  53013,  53026]) tensor([ 19,  19,  19,  19])
tensor([], dtype=torch.int64)
2 100 53015 64 2 2
softmax 직후... tensor([[-0.3256,  0.0208],
        [-0.3543,  0.1319],
        [-0.3649,  0.3069],
        [-0.0298, -0.1448]])
1개만 고른후... tensor([[ 1],
    